In [1]:
import h5py
import pandas as pd
import numpy as np
import os

In [2]:
def geo_fields_csv(raw_data_filepath):
    with h5py.File(raw_data_filepath, 'r') as hdf:

        year = []
        month = []
        day_of_month = []
        hour = []
        minute = []
        second = []
        latitude = []
        longitude = []
        precipRateNearSurface = []

        for i in range(3878):
            year.append(hdf['Geo_Fields']['Year'][i])
            month.append(hdf['Geo_Fields']['Month'][i])
            day_of_month.append(hdf['Geo_Fields']['DayOfMonth'][i])
            hour.append(hdf['Geo_Fields']['Hour'][i])
            minute.append(hdf['Geo_Fields']['Minute'][i])
            second.append(hdf['Geo_Fields']['Second'][i])
            latitude.append(hdf['Geo_Fields']['Latitude'][i][29][0])
            longitude.append(hdf['Geo_Fields']['Longitude'][i][29][0])
            precipRateNearSurface.append(hdf['SLV']['precipRateNearSurface'][i][29])
            
    geo_fields_data = pd.DataFrame({
        'Year' : year ,
        'Month' : month ,
        'DayOfMonth' : day_of_month ,
        'Hour' : hour ,
        'Minute' : minute ,
        'Second' : second ,
        'Latitude' : latitude ,
        'Longitude': longitude ,
        'precipRateNearSurface' : precipRateNearSurface
    })

    return geo_fields_data

In [16]:

folder_path = '/Volumes/陈炫羽的硬盘1/fy3g_data/2024_4/2024_401-407'
df_total = pd.DataFrame()
ls_filepaths=[]

for filename in os.listdir(folder_path):
    if filename.endswith('.HDF'):
        file_path = os.path.join(folder_path, filename)
        ls_filepaths.append(file_path)
print(ls_filepaths)
for i in range(0, len(ls_filepaths),2):
    file = ls_filepaths[i]
    print(file)
    df_new = geo_fields_csv(file)
    df_total = pd.concat([df_total, df_new], ignore_index=True)
    print(df_total.shape)




['/Volumes/陈炫羽的硬盘1/fy3g_data/2024_4/2024_401-407/FY3G_PMR--_ORBA_L2_KuR_MLT_NUL_20240401_0010_5000M_V0.HDF', '/Volumes/陈炫羽的硬盘1/fy3g_data/2024_4/2024_401-407/._FY3G_PMR--_ORBA_L2_KuR_MLT_NUL_20240401_0010_5000M_V0.HDF', '/Volumes/陈炫羽的硬盘1/fy3g_data/2024_4/2024_401-407/FY3G_PMR--_ORBA_L2_KuR_MLT_NUL_20240401_1403_5000M_V0.HDF', '/Volumes/陈炫羽的硬盘1/fy3g_data/2024_4/2024_401-407/._FY3G_PMR--_ORBA_L2_KuR_MLT_NUL_20240401_1403_5000M_V0.HDF', '/Volumes/陈炫羽的硬盘1/fy3g_data/2024_4/2024_401-407/FY3G_PMR--_ORBA_L2_KuR_MLT_NUL_20240401_1536_5000M_V0.HDF', '/Volumes/陈炫羽的硬盘1/fy3g_data/2024_4/2024_401-407/._FY3G_PMR--_ORBA_L2_KuR_MLT_NUL_20240401_1536_5000M_V0.HDF', '/Volumes/陈炫羽的硬盘1/fy3g_data/2024_4/2024_401-407/FY3G_PMR--_ORBA_L2_KuR_MLT_NUL_20240401_1709_5000M_V0.HDF', '/Volumes/陈炫羽的硬盘1/fy3g_data/2024_4/2024_401-407/._FY3G_PMR--_ORBA_L2_KuR_MLT_NUL_20240401_1709_5000M_V0.HDF', '/Volumes/陈炫羽的硬盘1/fy3g_data/2024_4/2024_401-407/FY3G_PMR--_ORBA_L2_KuR_MLT_NUL_20240401_1841_5000M_V0.HDF', '/Volumes/陈炫羽的硬盘1/f

In [21]:

df_total = df_total[df_total['Year'] >= 0]

# 按照 'DayOfMonth' 进行排序
df_total = df_total.sort_values('DayOfMonth')

# 按照 'Hour' 进行排序
df_total = df_total.sort_values('Hour')
df_total = df_total.sort_values('Minute')

# 将处理后的数据框保存为 CSV 文件
df_total.to_csv('FY3G_geodata_2024_401-407.csv', index=False)


In [4]:
filename1 = '/Volumes/陈炫羽的硬盘1/fy3g_data/2024_4/2024_401-407/FY3G_PMR--_ORBA_L2_KuR_MLT_NUL_20240401_0010_5000M_V0.HDF'
df_test = geo_fields_csv(filename1)

### 地图选择
在经过多方面考虑之后，我决定选择folium这个第三方库进行轨道的绘制。


In [5]:
import folium

In [10]:
import folium
import numpy as np

def create_shadow_area(latitudes, longitudes, width_km=5):
    # 地球半径（公里）
    earth_radius_km = 6371.0
    
    # 计算宽度对应的经纬度变化
    width_degree = width_km / earth_radius_km * (180 / np.pi)

    shadow_coordinates = []
    
    for i in range(len(latitudes) - 1):
        # 轨迹点
        lat1, lon1 = latitudes[i], longitudes[i]
        lat2, lon2 = latitudes[i + 1], longitudes[i + 1]
        
        # 随机生成两个偏移角度（左偏和右偏）
        angle1 = np.arctan2((lat2 - lat1), (lon2 - lon1)) + np.pi / 2
        angle2 = np.arctan2((lat2 - lat1), (lon2 - lon1)) - np.pi / 2
        
        # 计算偏移后的点
        shadow_point1 = (lat1 + width_degree * np.cos(angle1), lon1 + width_degree * np.sin(angle1))
        shadow_point2 = (lat1 + width_degree * np.cos(angle2), lon1 + width_degree * np.sin(angle2))
        
        shadow_coordinates.append(shadow_point1)
        shadow_coordinates.append(shadow_point2)

    return shadow_coordinates

latitude_test = df_test['Latitude'].tolist()
longitude_test = df_test['Longitude'].tolist()

# 创建地图
m = folium.Map(location=[0, 0], zoom_start=2)

# 绘制卫星轨迹
folium.PolyLine(locations=list(zip(latitude_test, longitude_test)), color='red').add_to(m)

# 创建阴影区域
shadow_area = create_shadow_area(latitude_test, longitude_test, width_km=5)

# 绘制透明阴影区域
folium.Polygon(locations=shadow_area, color='blue', fill=True, fill_opacity=0.3).add_to(m)

# 保存并显示地图
m.save("satellite_orbit_with_shadow.html")
